In [ ]:
import requests
from IPython.display import JSON

LOCAL = False

url = f"https://docker78406-saas-kit.jcloud.ik-server.com"

if LOCAL:
    import os
    from dotenv import load_dotenv

    load_dotenv()
    PORT = os.getenv("WEBBFF_SERVICE_PORT")

    url = f"http://localhost:{PORT}"

# Default headers (extended with authorization when necessary)
headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}

client = requests.session()

## Authentication service

### Login

In [ ]:
payload = {
    'email': 'simon@smartnuance.com',
    'password': 'f00bartest',
    'instance': 'smartnuance.com'
}
response = client.post(f'{url}/auth/login', json=payload, headers=headers)
print(response.status_code)

login_res = response.json()
JSON(login_res)


### Refresh tokens

In [ ]:
payload = {
    'refreshToken': login_res["refreshToken"],
}
response = client.post(f'{url}/auth/refresh', json=payload, headers=headers)
print(response.status_code)

refresh_res = response.json()
login_res["accessToken"] = refresh_res["accessToken"]
JSON(refresh_res)


### Revoke all tokens for a user

In [ ]:
payload = {
    'email': 'simon@smartnuance.com',
}
auth_header = {
    'Authorization': f'Bearer {login_res["accessToken"]}',
}
response = client.delete(f'{url}/auth/revoke/all', json=payload, headers=dict(headers, **auth_header))
print(response.status_code)


## Interact with event service

### Create workshop

In [ ]:
import random
titles = ['Yoga', 'Salsa', 'Martial arts', 'Jam session', 'Pilates', 'Zumba', 'Belly dance']
title = random.choice(titles)
payload = {
    'title': title,
    'slug': title.replace(' ', '-').lower(),
    'locationName': 'Ponto',
    'locationURL': 'https://goo.gl/maps/wE3viADXW1CB7LNP9',
}
auth_header = {
    'Authorization': f'Bearer {login_res["accessToken"]}',
    'role': 'event organizer',
    'instance': 'c5263570ono4ui8qfhgg',
}

response = client.put(f'{url}/event/workshop', json=payload, headers=dict(headers, **auth_header))
print(response.status_code)
workshop = response.json()
JSON(workshop)


### List workshops

In [ ]:
auth_header = {
    'Authorization': f'Bearer {login_res["accessToken"]}',
    'role': 'event organizer',
    'instance': 'c5263570ono4ui8qfhgg',
}
response = client.get(f'{url}/event/workshop/list', headers=dict(headers, **auth_header))
print(response.status_code)
workshops = response.json()["items"]
JSON(response.json())


### Delete workshop

In [ ]:
id = workshops[0]["id"]
print(f"delete {id}")

auth_header = {
    'Authorization': f'Bearer {login_res["accessToken"]}',
    'role': 'event organizer',
    'instance': 'c5263570ono4ui8qfhgg',
}
response = client.delete(f'{url}/event/workshop/{id}', headers=dict(headers, **auth_header))
print(response.status_code)
